## Extract API Data

In [ ]:
import requests
import pandas as pd
from datetime import datetime

In [ ]:
# Your API key from OpenWeather

API_KEY = "445f4f42617f805d6b2472725addf261"   
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"

In [ ]:
# List of cities to extract data for
cities = ["Delhi", "Bengaluru", "Hyderabad", "Jaipur",  "London", "New York", "Tokyo", "Sydney"]

In [ ]:
# Store Redponses

weather_data = []

for city in cities:
    params = {
        'q': city,
        'appid': API_KEY,
        'units' : 'metric'
    }
    response = requests.get(BASE_URL, params=params)


    if response.status_code == 200:
        data = response.json()
        weather_data.append({
        'city': data['name'],
        'temperature': data['main']['temp'],
        'humidity': data['main']['humidity'],
        'weather': data['weather'][0]['description'],
        'wind_speed': data['wind']['speed'],
        'timestamp': datetime.now()
    })
    else:
        print(f'Failed to get data for {city}: {response.status_code}')

# Convert to DataFrame
df_weather = pd.DataFrame(weather_data)
df_weather.head()

In [ ]:
#quick data check

df_weather.info()
df_weather.head()

## Load to MySQL

In [ ]:
import mysql.connector
from mysql.connector import connect

In [ ]:
# Connect to SQL
try:
    conn = mysql.connector.connect(
        host = 'localhost',
        user = 'root',
        password = 'your password',
        database = 'your database_name'
    )

    cursor = conn.cursor()
    print("Connected to SQL")
except Error as e:
    print("Error:", e)

In [ ]:
# Create tabe if not exisits
cursor.execute("""
create table if not exists weather_data(
    id int primary key auto_increment,
    city varchar(100),
    temperature float,
    humidity int,
    weather varchar(100),
    wind_speed float,
    timestamp datetime
)
""")

In [ ]:
# Insert Data

for _, row in df_weather.iterrows():
    cursor.execute("""
        insert ignore into weather_data(city, temperature, humidity, weather, wind_speed, timestamp)
        values(%s, %s, %s, %s, %s, %s)
    """,(
        row['city'],
        row['temperature'],
        row['humidity'],
        row['weather'],
        row['wind_speed'],
        row['timestamp']
    ))

conn.commit()
cursor.close()
conn.close()
print("Weather data successfuly loaded into MYSQL database.")

## SQL Queries for Weather Data Analysis

In [ ]:
# Reconnect to DB

try:
    conn = mysql.connector.connect(
        host = 'localhost',
        user = 'root',
        password = 'your password',
        database = 'your database_name'
    )

    cursor = conn.cursor()
    print("Connected to SQL")
except Error as e:
    print("Error:", e)

In [ ]:
# Query 1: Highest temperature
df_highest_temp = pd.read_sql("SELECT city, temperature FROM weather_data ORDER BY temperature DESC LIMIT 1", conn)
print("City with Highest Temperature: ")
print(df_highest_temp)

In [ ]:
# Query 2: Average Humidity

df_avg_humidity = pd.read_sql("SELECT AVG(humidity) AS avg_humidity FROM weather_data", conn)
print("\nAverage Humidity: ")
print(df_avg_humidity)

In [ ]:
# Query 3: Windy Cities

df_windy = pd.read_sql("SELECT city, wind_speed FROM weather_data WHERE wind_speed > 5 ORDER BY wind_speed DESC", conn)
print("\n Cities with Strong Winds:")
print(df_windy)

## Combining All The Result into a Single DataFrame and Transfer into CSV

In [ ]:
# Query 1: Highest temperature
df_highest_temp = pd.read_sql("SELECT city, temperature FROM weather_data ORDER BY temperature DESC LIMIT 1", conn)
df_highest_temp.insert(0, 'query', 'Highest Temp')

# Query 2: Average Humidity

df_avg_humidity = pd.read_sql("SELECT AVG(humidity) AS avg_humidity FROM weather_data", conn)
df_avg_humidity.insert(0, 'query', 'Avg Humidity')

# Query 3: Windy Cities

df_windy = pd.read_sql("SELECT city, wind_speed FROM weather_data WHERE wind_speed > 5 ORDER BY wind_speed DESC", conn)
df_windy.insert(0, 'query', 'Strong Winds')

# Combine all Results
df_combined = pd.concat([df_highest_temp, df_avg_humidity, df_windy], ignore_index = True)

#Save to single CSV
df_combined.to_csv("weather_analysis.csv", index = False)

conn.close()
print("All query result saved to 'weather_analysis.csv'")
